# DINO — Self-supervised representation learning (with segmentation capabilities)
[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи](https://www.youtube.com/watch?v=h3ij3F3cPIk)

[vision_transformer](https://github.com/google-research/vision_transformer)

Для начала подгрузим модель DINO (self-**DI**stillation with **NO** labels)



In [ ]:
# fix random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/facebookresearch/dino.git

## Сегментация изображений

Теперь загрузим случайную картинку (можно выбрать любую, просто замените ссылку на свою)

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/capybara_image.jpg'
!wget $URL -qO test.jpg

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

input_img = Image.open('/content/test.jpg')
input_img.resize((400,300))

Получим результат с помощью DINO

In [ ]:
from IPython.display import clear_output
!python /content/dino/visualize_attention.py --image_path /content/test.jpg 
clear_output()

Посмотрим на картинки, которые генерирует DINO

In [ ]:
from glob import glob

def img_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(28,8))
    for num, img in enumerate(imgs):
        img_PIL = Image.open(img)
        ax[num].imshow(img_PIL)
        ax[num].set_xticks([])
        ax[num].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

img_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)
plt.show()

Мы видим 6 карт внимания (self-attention maps - веса слоя self-attention) на 6 головах Visual Transformer. В результате self-supervised обучения по методике DINO, трансформер САМОСТОЯТЕЛЬНО придумал обращать внимание на различные части изображения, таким образом, производя семантическую сегментацию.

Мы можем все эти карты внимания объединить в одно изображение, и просто назначить каждой карте свой цвет, а в качестве прозрачности использовать интенсивность

In [ ]:
import matplotlib.colors as mcolors
from matplotlib import cm

def overlay(img, segmentations):
    img_PIL = Image.open(img)
    fig, ax = plt.subplots(ncols=2, figsize=(10,10))
    ax[0].imshow(img_PIL)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    
    ax[1].imshow(img_PIL.convert('LA'), alpha=0.5)
    for num, img in enumerate(segmentations):
        segment_PIL = Image.open(img).convert('LA')
        segment_arr = np.array(segment_PIL)
        colors = [(*cm.tab10(num)[:-1], c) for c in np.linspace(0,0.75,100)]
        cmap = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
        ax[1].imshow(segment_arr[:,:,0], cmap=cmap)
    ax[1].set_facecolor('black')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

overlay('/content/img.png', sorted(glob('*.png'))[:-1])
plt.show()

Видим, что DINO сегментирует разные части нашей картинки на разные семантические группы. В случае с капибарой - это голова, лицо (нос, глаза) и тело.

## Сегментация видео

Вы думали, что на этом все? Нет, DINO может еще удивить. Например, она умеет сегментировать видео

In [ ]:
!sudo pip install --upgrade youtube_dl

Скачаем видео (можно любое)

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/cheetah_video.mp4'
!youtube-dl -o video.mp4 $URL

И сгенерируем сегментированное видео

In [ ]:
from IPython.display import clear_output
!python /content/dino/video_generation.py --input_path /content/video.mp4 --output_path  /content/video_segmented --resize 360 640
!ffmpeg -i /content/video.mp4 -vf scale=640:360 /content/video_scaled.mp4
!ffmpeg \
  -i /content/video_scaled.mp4 \
  -i /content/video_segmented/video.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map '[vid]' \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4

clear_output()
print('Complete')

Посмотрим, что получилось. Обратите внимание, эта сеть обучалась в режиме self-supervision!

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Кластеризация

А еще DINO умеет кластеризовать изображения. Выполнять не будем, так как процесс не быстрый, но можем посмотреть на результаты из их статьи

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/clustering_dino.gif" width="400">

[Advancing the state of the art in computer vision with self-supervised Transformers and 10x more efficient training](https://ai.facebook.com/blog/dino-paws-computer-vision-with-self-supervised-transformers-and-10x-more-efficient-training/)

#### Pixel-wise cross entropy loss


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/img_license/pixel_wise_cross_entropy_loss.png" width="800">